# Introduction

Word2Vec is a recently developed family of algorithms for generating distributed representations of words from a large corpus of text. These distributed representations are sometimes referred to as 'word embeddings' because the algorithms that generate them are designed to embed a vocabulary of words in a relatively low dimensional vector space (i.e. the dimensionality of the embeddings is usually on the order of hundreds, while the size of the vocabulary is usually on the order of tens or hundreds of thousands). Well known methods for creating word embeddings include LSA, HAL, BEAGLE, GloVe, and Word2Vec. Word2Vec's popularity as an embedding model is due to the fact that the representations it produces are able capture interesting relationships involving multiple words. 

Interestingly, Word2Vec uses two distinct model architectures: the CBOW (continuous-bag-of-words) architecture and the Skip-gram architecture. To explain, the CBOW architecture learns to predict a target word given a 'bag' or context of surrounding words. The Skip-gram architecture, on the other hand, does exactly the opposite - it learns to predict a bag of surrounding words given a target word. The figure below, taken from Mikolov et al. (2013), provides a nice visual depiction of each architecture, where $w$ indicates a word, and $t$ indicates the target word position.  

<br>
<center><img src="images/cbow.png" width=500px></center>  
<br>

The rest of this notebook describes each model in some mathematical detail, drawing on explanations provided by Rong (2015). The notebook also provides example applications of each algorithm to a corpus of approximately 5,000 wikipedia articles. Optimizations concerning the use of hierarchical softmax and negative sampling are also discussed. 

# Part 1: Continuous Bag of Words (CBOW)

As illustrated in the figure above, the CBOW architecture is akin to a three layer neural network. The input layer encodes a binary vector indicating a bag of words (order is not important, so the $w(t-1)...$ notation in the figure merely indicates the window around the target word from which the bag is drawn), and the hidden layer maps the input layer into a low dimensional space without applying a non-linearity. Finally, the hidden layer is mapped to a softmax output layer that gives a probability distribution over the words in the model's vocabulary. 

In the simplest case, the input is a single word drawn from a corpus, and the target is the next word in the corpus. The input is encoded as a "one-hot" vector, and hence it extracts a single column from the input-to-hidden weight matrix ($W_1$). As such the value of the hidden layer is $h = W_1 x$, where $x$ is the input vector. Next, each unit in the output layer encodes a dot product between $h$ and a row in the hidden-to-output weight matrix ($W_2$). The softmax function is then used to convert these dot products into a proper probability distribution. It is important to highlight that each column of $W_1$ is vector representation of a particular word, as is each row of $W_2$. (So each word has *two* vector representations). The goal of learning is to make the $W_1$ vector representation of a word most similar to the $W_2$ vector representation of the word that is most likely to follow it in the corpus. 

Alternatively, one can think of the model as performing a variant of logistic regression using a factored weight matrix. The hidden layer does not include an element-wise non-linearity, so a weight matrix between the input and output layers is factored into two $N x V$ and $V x N$ components, where $V$ is the size of the vocabulary, and $N$ is the dimensionality of the hidden layer. These component matrices each contain a distributed representation of each word in the vocabulary - call them $v_w$ and $v_{w}^{'}$ for word $w$. So, the activation of the output layer after applying the softmax is as follows for input word $i$ and output word $o$:

$y_o = \frac{e^{v_i^{T} v_o^{'}}}{\sum_{j=1}^{V} e^{v_i^{T} v_j^{'}}}$

Learning is conducted by applying gradient descent to minimize a cost function defined in terms of the negative log-likelihood of the correct output word $o$. This cost function is similar to the one used previously for multiclass logistic regression (see the notebook on backpropogation):

$J(\theta) = -log(\frac{e^{v_i^{T} v_o^{'}}}{\sum_{j=1}^{V} e^{v_i^{T} v_j^{'}}}) = - v_i^{T} v_o^{'} + log\sum_{j=1}^{V} e^{v_i^{T} v_j^{'}}$  

where again $j$ is the index of the correct output word, $i$ is the index of the input word, and $V$ is the size of the vocabulary. Differentiating this cost function with respect to the total input to each output unit gives the familiar softmax derivative:

$ \frac{\partial J(\theta)}{\partial v_i^{T} v_j^{'}} = \sum \limits_j \frac{\partial J(\theta)}{\partial y_j} \frac{\partial y_j}{\partial v_i^{T} v_j^{'}} =  y_j - t_j $

where $t_k$ is the target output for unit $k$. With this derivative, we can compute the gradient of the weights in the network using backprogation. At an intuitive level, the updates to $W_2$ are easy to understand. Each row $j$ in $W_2$ is updated with values of the hidden layer activities, but scaled by both the learning rate and the prediction error at $y_k$ (i.e. the softmax derivative). So, for a given output unit, if the model incorrectly guesses that the word corresponding to this unit ought to be predicted, the unit's incoming weights will be decremented to be more *dissimilar* to the hidden layer vector. In effect, this lowers the value of the dot product between the first layer representation of the input word, and the second layer representations of the output word under consideration. On the other hand, if the model incorrectly fails to predict the correct word, the unit corresponding to this word will have its incoming weights incremented to become more *similar* to the hidden layer vector. This raises the value of the dot product between the first layer representation of the input word and the second layer representation of the correct output word. As such, when same input is provided again to the model, it will be more likely to predict the correct output. The updates to $W_1$ follow a similar but less directly interpretable pattern. 

With a large-sized vocabulary, CBOW is actually quite slow to train, mostly due to the computation of the softmax function on the output layer. We can see this by training a model on a few thousand documents from wikipedia. Training is performed by attempting to predict each word in each sentence of every document from a bag of up to six surrounding words (generalizing to multiple input words does not change any of the math - the hidden layer just ends up encoding a sum of the first layer representations of each input word). 

To give a demonstration, the first thing to do is to build a vocabulary by getting word counts from a collection of documents. Words with low counts will be ignored to avoid idiosyncratic mispellings and extremely rare words; the remaining words will form the vocabulary. 

Counting words can be sped up using python's multiprocessing library as follows:

In [5]:
import collections
import multiprocessing
from utils import docstream, countwords

counts = collections.Counter()  
pool = multiprocessing.Pool()

# Stream n files from wikipedia dump and count words in parallel
for dlist in docstream(size=100):
    results = pool.map_async(countwords, dlist)
    for r in results.get():
        counts.update(r)
 
vocab = [x[0] for x in counts.iteritems() if x[1] > 25]
print 'Total Number of Words: ', sum(counts.values())
print 'Vocabulary Size: ', len(vocab)

Total Number of Words:  15738309
Vocabulary Size:  27263


Now we can define the CBOW model using a variation on the neural network class used in other notebooks. Some helper functions for grabbing contexts from sentences and converting words into one-hot vectors have been defined elsewhere to keep things manageably concise. We'll get access to these functions by inheriting from an EmbeddingModel class as follows:

In [6]:
import numpy as np
import operator
import embedding # Functions for get contexts from sentences etc.

class CBOW(embedding.EmbeddingModel):
    """
    A shallow neural network that implements the 'continous bag of words' 
    learning algorithm described in Mikolov et al (2013). 
    
    Parameters:
    -----------
    voc : dict
        The vocabulary of words that embeddings are being learned for.
    dim : int 
        The dimensionality of the hidden layer of the network.
    eps : float, optional
        Scaling factor on random weight initialization. By default, the 
        weightsare chosen from a uniform distribution on the interval 
        [-0.1, 0.1].
    """
    def __init__(self, vocab, dim, eps=0.15):
        self.w1 = np.random.random((dim, len(vocab)))*eps*2-eps
        self.w2 = np.random.random((len(vocab), dim))*eps*2-eps
        self.word_indices = {j:i for i,j in enumerate(vocab)}
        self.win_size = 3
        self.vocab = vocab
            
    def get_activations(self, x):
        self.yh = np.dot(self.w1, x.T)
        self.yo = self.softmax(np.dot(self.w2, self.yh))

    def train(self, ndocs, rate=0.3): 
        for xs, ys in self.batch_generator(ndocs):
            bsize = float(xs.shape[1])
            # Compute activations
            self.get_activations(xs.T)

            # Compute gradients               
            yo_grad = self.yo-ys
            yh_grad = np.dot(self.w2.T, yo_grad)

            w1_grad = np.dot(yh_grad, xs.T) / bsize
            w2_grad = np.dot(yo_grad, self.yh.T) / bsize

            # Update weights
            self.w1 += -rate * w1_grad
            self.w2 += -rate * w2_grad
        
        # Create word vectors from network weights
        self.word_vecs = self.w1.T + self.w2
        norms = np.linalg.norm(self.word_vecs, axis=1)
        self.word_vecs = np.divide(self.word_vecs, norms[:, np.newaxis])

In [ ]:
embedder = CBOW(vocab, dim=150) 
embedder.train(ndocs=5000)

In [8]:
query_list = ['small','roman','medieval','cash','car','waves','french','english','texas']
for query in query_list:
    print 'Nearest neighbors to "%s":' %query
    embedder.get_synonyms(query)
    print ''


Nearest neighbors to "small":

small 1.0
large 0.692444277791
tiny 0.660107399535
larger 0.653754282846
smaller 0.583412575441

Nearest neighbors to "roman":

roman 1.0
catholic 0.673838975686
byzantine 0.643910891448
catholicism 0.60339234428
empire 0.60302562371

Nearest neighbors to "medieval":

medieval 1.0
renaissance 0.701334013228
modern 0.604067449688
iconography 0.598731651187
hellenistic 0.594439571047

Nearest neighbors to "cash":

cash 1.0
payments 0.657618525796
payment 0.655440486
banks 0.622166553811
loans 0.612372415847

Nearest neighbors to "car":

car 1.0
accident 0.68987610779
crash 0.608334304968
injured 0.60722705516
drunk 0.597927025079

Nearest neighbors to "waves":

waves 1.0
wave 0.617193281391
propagate 0.607981741084
seismic 0.568776460093
light 0.531950285785

Nearest neighbors to "french":

french 1.0
italian 0.70883228134
english 0.707912943792
german 0.689125854016
norwegian 0.634899206725

Nearest neighbors to "english":

english 1.0
german 0.71134702388

# Part 2: Skip-gram

Content in progress...